In [0]:
import requests

Funções nb_deputados

In [0]:
def get_deputados(pagina=1, itens=100):
    """
    Requisição para API
    pagina: número da página
    itens: número de itens por página (máx 100)
    Retorna a lista de deputados
    """
    url = f"https://dadosabertos.camara.leg.br/api/v2/deputados?pagina={pagina}&itens={itens}"
    response = requests.get(url, headers={"accept": "application/json"})
    response.raise_for_status()
    return response.json()["dados"]


def get_all_deputados(itens_por_pagina=100, limite_paginas=None):
    """
    Busca todos os deputados da API (com paginação).
    itens_por_pagina: máximo de registros por página (default 100)
    limite_paginas: Quais páginas, onde None = todas disponíveis
    lista todos os deputados
    """
    deputados = []
    pagina = 1
    while True:
        dados = get_deputados(pagina=pagina, itens=itens_por_pagina)
        if not dados:  
            break
        deputados.extend(dados)
        pagina += 1
        if limite_paginas and pagina > limite_paginas:
            break
    return deputados


def deputados_to_dataframe(deputados):
    """
    Converte lista de deputados para DataFrame Spark.
    """
    return spark.createDataFrame(deputados)


def save_deputados(df, tabela="tb_deputados"):
    """
    Salva o DataFrame como tabela no Databricks.
    tabela: Nome da tabela a ser criada
    """
    df.write.mode("overwrite").saveAsTable(tabela)
    print(f"Tabela '{tabela}' criada/atualizada com sucesso!")